# `custom_tools.py`

In [1]:
%run custom_tools.py

In [2]:
print_md('**Test markdown**', 'green')
print_md('\t test_1    \n\
          \t *test_2*  \n\
          \t ~~test_3~~', 'blue')

<span style="color:green">**Test markdown**</span>

<span style="color:blue">&emsp; test_1    <br>          &emsp; *test_2*  <br>          &emsp; ~~test_3~~</span>

In [3]:
print_count(5, 12)

5/12 (41.67%)


In [4]:
unlist([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [14]:
# read the first 1000 rows of impact
impact = pd.read_csv('../../data/annotated_final_IMPACT_mutations_180508.txt', sep='\t', nrows=1000)

get_table(impact['VEP_VARIANT_CLASS'])

,count_,freq_
SNV,839,83.90%
deletion,105,10.50%
insertion,41,4.10%
indel,8,0.80%
substitution,7,0.70%


# Other

In [ ]:
%run ../setup_environment.ipy
%run ../Selene_Job.ipy